# Object GUI

## Choose the working directory

In [1]:
from ipyfilechooser import FileChooser
import os

# Create new FileChooser:
# Path: current directory
# Show hidden files: no
# Use the default path and filename as selection: yes
# Use folder icons: yes
fdialog = FileChooser(
    os.getcwd(),
    filename='',
    title='<b>Change Directory</b>',
    show_hidden=False,
    select_default=True,
    use_dir_icons=True
)

display(fdialog)

# Callback func
def change_directory(chooser):
    os.chdir(chooser.selected)
    print("Change directory to: " + os.getcwd())

# Register callback function
fdialog.register_callback(change_directory)


FileChooser(path='/Users/chris/Documents/GitHub/ObjectGUI', filename='', show_hidden='False')

## Choose Directory

In [2]:
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import display
import glob

fdialog2 = FileChooser(
    os.getcwd(),
    filename='',
    title='',
    change_desc = 'Choose',
    show_hidden=False,
    select_default=True,
    use_dir_icons=True
)

display(fdialog2)

module_list = {}
module_sel = widgets.Select()
module_sel.options = module_list.keys()
display(module_sel)

FileChooser(path='/Users/chris/Documents/GitHub/ObjectGUI', filename='', show_hidden='False')

Select(options=(), value=None)

## Object

In [3]:
# create initial widget
object_sel = widgets.Select()
object_sel.options = []
object_sel.rows=10
display(object_sel)

# Callback func
def search_module(chooser):
    cw = os.getcwd()
    os.chdir(chooser.selected)
    module_name = os.path.basename(os.path.normpath(chooser.selected))
    global module_list
    class_list = []
    # look for Module name folder
    if bool(glob.glob(module_name)):
        os.chdir(module_name)
        if bool(glob.glob("__init__.py")):
            ini_file = open("__init__.py")
            for line in ini_file.readlines():
                # skip empty lines
                line = line.strip()
                if not line:  # line is blank
                    continue
                last_word = line.split()[-1]
                if(last_word[0].isupper()):
                    class_list += [last_word]
            ini_file.close()
    # change possible selections
    module_list[module_name] = class_list
    module_sel.options = module_list.keys()
    module_sel.value = module_name
    display(module_sel)
    os.chdir(cw)

# Register callback function
fdialog2.register_callback(search_module)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        module_name = change['new']
#         object_sel.value = None
        object_sel.options = module_list[module_name]
        display(object_sel)

module_sel.observe(on_change)

Select(options=(), rows=10, value=None)

## Constructor Options

In [4]:
import importlib

text_display = widgets.VBox([])
display(text_display)

def on_select_object(change):
    if change['type'] == 'change' and change['name'] == 'value':
        object_name = change['new']
        module_import = importlib.import_module(module_sel.value)
        object_import = getattr(module_import, object_name)
        argsList = getattr(object_import, 'argsList', 'None')
        text_list = []
        if argsList != 'None':
            for key in argsList:
                if type(key[1]) == str:
                    a = widgets.Text(value = key[1],
                                 placeholder='Type something',
                                 description= key[0],
                                 disabled=False)
                    text_list += [a]
                elif type(key[1]) == int:
                    a = widgets.IntText(
                                value=key[1],
                                description=key[0],
                                disabled=False)
                    text_list += [a]
                elif type(key[1]) == bool:
                    a = widgets.Checkbox(
                                value=key[1],
                                description=key[0],
                                disabled=False)
                    text_list += [a]
                else:
                    a = widgets.Text(value = str(key[1]),
                                 placeholder='Type something',
                                 description= key[0],
                                 disabled=False)
                    text_list += [a]
        text_display.children=tuple(text_list)     

object_sel.observe(on_select_object)

updated_arg_list = {}
button = widgets.Button(description='Done')
out = widgets.Output()

def on_button_clicked(_):
    arg = text_display.children
    with out:
        for a in arg:
            global updated_arg_list
            updated_arg_list = {}
            updated_arg_list[a.description]=a.value
            display(updated_arg_list)
            
button.on_click(on_button_clicked)            
display(button)
display(out)

VBox()

Button(description='Done', style=ButtonStyle())

Output()